In [11]:
import numpy as np
import scipy.io
train_mat = scipy.io.loadmat('train_data.mat') 
test_mat = scipy.io.loadmat('test_data.mat') 

In [65]:
def get_data_from_mat(train_file,test_file):
    train_mat = scipy.io.loadmat(train_file) 
    test_mat = scipy.io.loadmat(test_file) 
    print(train_mat.keys())
    print(test_mat.keys())
    train_np = np.array(train_mat['train_data']).transpose(2,0,1)
    #print(train_np.size)
    test_np = np.array(test_mat['test_data']).transpose(2,0,1)
    print('Train shape: ',train_np.shape)
    print('Test shape: ',test_np.shape)
    return  train_np, test_np


train_data, test_data = get_data_from_mat('train_data.mat','test_data.mat')

dict_keys(['__header__', '__version__', '__globals__', 'train_data'])
dict_keys(['__header__', '__version__', '__globals__', 'test_data'])
Train shape:  (112, 90, 90)
Test shape:  (28, 90, 90)


In [13]:
print(train_data[0])

[[0.         0.         0.13152513 ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.13152513 0.         0.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 0.         0.         0.28060623]
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.         0.         0.         ... 0.28060623 0.         0.        ]]


In [26]:
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact

def visualize(image):  
    plt.figure("sample", (12, 6))
    plt.subplot(1, 2, 1)
    plt.imshow(image, cmap="gray")    
    plt.subplot(1, 2, 2)
    plt.imshow(image, cmap="gray")
    plt.show()      


#@interact
def show_ds(ds):
    print("ds shape:",ds.shape)
    print("ds max:",np.max(ds))
    print("ds min:",np.min(ds))
    print("ds average:",np.average(ds))
    print("ds length:", len(ds))
    @interact
    def visualize_set(scan_index=(0,len(ds)-1)):
        print(scan_index)
        visualize(ds[scan_index,:,:])

show_ds(train_data)

ds shape: (112, 90, 90)
ds max: 1.0
ds min: 0.0
ds average: 0.024116160549305543
ds length: 112


interactive(children=(IntSlider(value=55, description='scan_index', max=111), Output()), _dom_classes=('widget…

In [27]:
show_ds(test_data)

ds shape: (28, 90, 90)
ds max: 1.0
ds min: 0.0
ds average: 0.024994026906171023
ds length: 28


interactive(children=(IntSlider(value=13, description='scan_index', max=27), Output()), _dom_classes=('widget-…

# Preparação do dataset

In [68]:
from torch.utils.data import Dataset
import numpy as np

In [80]:
#Transformar os dados de treino de modo a que cada uma das matrizes ficam em vetor
train_data, test_data = get_data_from_mat('train_data.mat','test_data.mat')

arr = []
for line in train_data:
    arr.append((np.array(line)).flatten())
train_data_flat = np.array(arr)
print('Shape dos dados de treino depois da transformação: ', train_data_flat.shape)

dict_keys(['__header__', '__version__', '__globals__', 'train_data'])
dict_keys(['__header__', '__version__', '__globals__', 'test_data'])
Train shape:  (112, 90, 90)
Test shape:  (28, 90, 90)
Shape dos dados de treino depois da transformação:  (112, 8100)
